# Imports and Setup

## Asynchronous Execution

In [1]:
import threading

In [ ]:
# multithreading lock to pause experiment loop during user feedback collection
experiment_lock = threading.Event()
experiment_lock.set()

## Graphical User Interface

In [ ]:
import ipywidgets as ipw
import matplotlib.pyplot as plt
import ternary

# enable ipympl
%matplotlib widget

# turn on interactive mode so that plots only appear where we want it to
plt.ioff()

In [ ]:
# create the gui elements
fig, tax = ternary.figure()

pause_button = ipw.Button(description='Pause')
undo_button = ipw.Button(description='Undo Point')
continue_button = ipw.Button(description='Continue')

output = ipw.Output()

In [ ]:
# store a list of points where the user has clicked
points = []

In [ ]:
def draw_background_plot(tax):
    '''
    Plot the data from the current experiment loop.

    Parameters
    ----------
    tax: ternary axes on which to plot the data
    '''

    pass

In [ ]:
def draw_user_points(ax):
    '''
    Draw the points where the user has clicked.

    Parameters
    ----------
    ax: the axes on which to draw the points
    '''

    # create separate x and y coordinate lists for every point in points
    x, y = zip(*points)

    ax.plot(x, y, 'k*-')

    plt.draw()

In [ ]:
def plot_callback(event):
    '''
    Store the point where the user clicked and add it to the plot.

    Parameters
    ----------
    event: a structure which has information about the mouse event
    '''
    
    if not experiment_lock.is_set():
        # add the mouse position (in data coordinates) to the points list
        points.append([event.xdata, event.ydata])

        draw_user_points(tax.get_axes())

cid = fig.canvas.mpl_connect('button_press_event', plot_callback)

In [ ]:
def pause_callback(button):
    '''
    Set the internal flag of the threading Event object to false.

    Parameters
    ----------
    button: the instance of button that was clicked
    '''
    
    if experiment_lock.is_set():
        output.append_stdout('user has paused')
        experiment_lock.clear()

pause_button.on_click(pause_callback)

In [ ]:
def continue_callback(button):
    '''
    Set the internal flag of the threading Event object to true

    Parameters
    ----------
    button: the instance of button that was clicked
    '''
    
    if not experiment_lock.is_set():
        output.append_stdout('user has unpaused')
        experiment_lock.set()

continue_button.on_click(continue_callback)

In [ ]:
# assemble the gui elements
buttons = ipw.HBox([pause_button, undo_button, continue_button])
gui = ipw.VBox([fig.canvas, buttons, output])

## CAMEO

In [ ]:
# get the simulation test data
# !wget -O CAMEO2_support_files_220104a.zip https://drive.google.com/u/0/uc?id=1UnKijzN_6shj-T2r37Jm2V6PSqTJMO-6&export=download
# !unzip -o CAMEO2_support_files_220104a.zip

In [ ]:
import numpy as np
from cameo import *
from scipy.io import loadmat
from scipy.stats import entropy
from ternary.helpers import project_sequence

## Experiment Loop

In [2]:
def main(output):
    '''
    Run the calculation loop while checking if the user is waiting to
    provide input. If so, pause the loop, collect input, then resume.

    Parameters
    ----------
    output: the ipywidgets Output widget to print messages
    '''

    data = loadmat('FeGaPd_full_data_220104a.mat')

    # composition data in cartesian coordinates
    composition = data['C']
    idx = [1,2,0]
    cartesian = np.array(list(zip(*project_sequence(composition[:,idx]))))

    # x ray diffraction data
    xrd = data['X'][:,631:1181] 
    two_theta = data['T'][:,631:1181]

    # true and density functional theory predicted phase labels
    true_labels = data['labels_col']
    dft_labels = data['labels_DFT'].flatten()

    remnant_magnetization = data['Mag_modified']

    num_measurements = 20
    num_clusters = 5
    num_total = cartesian.shape[0]

    measured = np.random.permutation(num_total)[:num_measurements]
        
    for _ in range(10):
        unmeasured = np.setdiff1d(np.arange(0, num_total), measured)

        labels, label_probs = phase_mapping(xrd[measured, :], num_clusters)

        y_mean, y_var, f_mean, f_var = gpc_phasemapping(cartesian[measured, :],
                                                        labels,
                                                        cartesian,
                                                        num_clusters)

        gpc_labels = y_mean.argmax(axis=1)

        acquisition = entropy(y_mean, axis=1)

        next_sample = np.argmax(acquisition)

        measured = np.append(measured, next_sample)

        if not experiment_lock.is_set():
            experiment_lock.wait()
    
    output.append_stdout('all done')


# Putting It Together

In [ ]:
# begin the experiment in a new thread
experiment_thread = threading.Thread(target=main,
                                     args=(output,),
                                     name='experiment')
experiment_thread.start()

In [ ]:
# start the gui
gui